In [1]:
from bs4 import BeautifulSoup
import requests as re
import pandas as pd
import numpy as np
import os
import io
import csv
from datetime import datetime
import logging
logging.getLogger().setLevel(logging.INFO)

# https://cloud.google.com/sql/docs/postgres/connect-external-app#languages
import psycopg2
conn = psycopg2.connect(user='airflow', password='Xypherium-0',
                        dbname='jpstat',
                        host='35.224.240.50')

from google.cloud import storage
bucket_name = "i-agility-212104.appspot.com"
mdir = os.path.join('extracts', 'monthly_reports')

In [2]:
#def files_review_task():
    

#Retrieve all months currently available 
main_table = []
#url = input("http://www.e-stat.go.jp/SG1/estat/OtherListE.do?bid=000001006005&cycode=1")
url = "http://www.e-stat.go.jp/SG1/estat/OtherListE.do?bid=000001006005&cycode=1"
path='https://www.e-stat.go.jp'
r = re.get(url)
data = r.text
year='None'
soup = BeautifulSoup(data, "lxml")
table = soup.find('div', {'class': 'stat-cycle_sheet'})
for year_section in table.find_all('ul', {'class': 'stat-cycle_ul_other'}):
    header = year_section.find('li', {'class': 'stat-cycle_header'})
    year = header.find('span').get_text(' ', strip=True)
    #   print('year changed to ' + year)
    months_section = year_section.find('li', {'class': 'stat-cycle_item'})
    for month_row in months_section.find_all('div'):
        month_a = month_row.find('a')
        month = month_a.get_text().rstrip('.\n')
        monthurl = path + month_a.get('href')

        main_table.append({
            'year': year,
            'month': month,
            'url': monthurl
        })

top_df = pd.DataFrame(main_table)

logging.info('DataFrame of months generated: ' + str(top_df.size) + ' months available, from ' 
    + top_df.loc[len(top_df)-1,'year'] + ' ' + top_df.loc[len(top_df)-1,'month'] + ' to ' 
    + top_df.loc[0,'year'] + ' ' + top_df.loc[0,'month'])



INFO:root:DataFrame of months generated: 456 months available, from 2006 Dec to 2018 Jan


In [4]:
retrieve_month_excels_starttime = datetime.utcnow()
logging.info('Starting to retrieve urls of every excel sheet at ' + str(retrieve_month_excels_starttime) + ' UTC.')

excels_df = pd.concat([retrieve_month_excels(murl, path) for murl in top_df['url']])
excelref_df = top_df.merge(excels_df, how='left', on='url')

logging.info('Excel URLs retrieved in ' + str(datetime.utcnow() - retrieve_month_excels_starttime) + '.')
logging.info(str(len(excelref_df)) + ' files to retrieve. Writing table to database...')

INFO:root:Starting to retrieve urls of every excel sheet at 2018-10-09 03:45:24.801722 UTC.
INFO:root:Retrieved excel URLs from month-URL: https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20180&month=11010301&result_back=1.
INFO:root:Retrieved excel URLs from month-URL: https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20180&month=11010302&result_back=1.
INFO:root:Retrieved excel URLs from month-URL: https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20180&month=11010303&result_back=1.
INFO:root:Retrieved excel URLs from month-URL: https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20180&month=12040604&result_back=1.
INFO:root:Retrieved excel URLs from month-URL: https

INFO:root:Retrieved excel URLs from month-URL: https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20150&month=23070907&result_back=1.
INFO:root:Retrieved excel URLs from month-URL: https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20150&month=23070908&result_back=1.
INFO:root:Retrieved excel URLs from month-URL: https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20150&month=23070909&result_back=1.
INFO:root:Retrieved excel URLs from month-URL: https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20150&month=24101210&result_back=1.
INFO:root:Retrieved excel URLs from month-URL: https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=00000

INFO:root:Retrieved excel URLs from month-URL: https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20120&month=24101210&result_back=1.
INFO:root:Retrieved excel URLs from month-URL: https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20120&month=24101211&result_back=1.
INFO:root:Retrieved excel URLs from month-URL: https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20120&month=24101212&result_back=1.
INFO:root:Retrieved excel URLs from month-URL: https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20110&month=11010301&result_back=1.
INFO:root:Retrieved excel URLs from month-URL: https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=00000

INFO:root:Retrieved excel URLs from month-URL: https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20080&month=11010301&result_back=1.
INFO:root:Retrieved excel URLs from month-URL: https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20080&month=11010302&result_back=1.
INFO:root:Retrieved excel URLs from month-URL: https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20080&month=11010303&result_back=1.
INFO:root:Retrieved excel URLs from month-URL: https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=000000070001&tclass1=000001008739&year=20080&month=12040604&result_back=1.
INFO:root:Retrieved excel URLs from month-URL: https://www.e-stat.go.jp/en/stat-search/files?cycle=1&layout=datalist&toukei=00200523&tstat=00000

In [ ]:
tempdf = excelref_df.copy()
tempdf['updated_on'] = retrieve_month_excels_starttime.strftime('%Y-%m-%d')
tempdf.head()

In [5]:
import pandas.io.sql as sqlio
sql = "SELECT year, month, url, excel_num, excel_description, excel_url FROM public.jpstat_excel_urls;"
curr_table = sqlio.read_sql_query(sql, conn)
curr_table.head()

,year,month,url,excel_num,excel_description,excel_url
0,2018,Jan,https://www.e-stat.go.jp/en/stat-search/files?..,0,"Number of Intra-prefectural Migrants, In-migr....",https://www.e-stat.go.jp/en/stat-search/file-d...
1,2018,Jan,https://www.e-stat.go.jp/en/stat-search/files?...,1,"Number of Intra-prefectural Migrants, In-migra...",https://www.e-stat.go.jp/en/stat-search/file-d...
2,2018,Jan,https://www.e-stat.go.jp/en/stat-search/files?...,2,Number of Inter-prefectural Migrants by Sex an...,https://www.e-stat.go.jp/en/stat-search/file-d...
3,2018,Jan,https://www.e-stat.go.jp/en/stat-search/files?...,3-1,Number of In-migrants from Other Prefectures b...,https://www.e-stat.go.jp/en/stat-search/file-d...
4,2018,Jan,https://www.e-stat.go.jp/en/stat-search/files?...,3-2,Number of Out-migrants to Other Prefectures by...,https://www.e-stat.go.jp/en/stat-search/file-d...


In [15]:
disjoint_df = pd.concat([curr_table, excelref_df]).drop_duplicates(keep=False, subset=['excel_url'])
disjoint_df = pd.merge(disjoint_df, excelref_df, how='inner')
disjoint_df

C:\Users\friedemann.ang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


,excel_description,excel_num,excel_url,month,url,year
0,"Number of Intra-prefectural Migrants, In-migr...",1,https://www.e-stat.go.jp/en/stat-search/file-d...,Jul,https://www.e-stat.go.jp/en/stat-search/files?...,2018
1,Number of Inter-prefectural Migrants by Sex a...,2,https://www.e-stat.go.jp/en/stat-search/file-d...,Jul,https://www.e-stat.go.jp/en/stat-search/files?...,2018
2,Number of In-migrants from Other Prefectures ...,3-1,https://www.e-stat.go.jp/en/stat-search/file-d...,Jul,https://www.e-stat.go.jp/en/stat-search/files?...,2018
3,Number of Out-migrants to Other Prefectures b...,3-2,https://www.e-stat.go.jp/en/stat-search/file-d...,Jul,https://www.e-stat.go.jp/en/stat-search/files?...,2018
4,Number of Net-migration by Age (Five-Year Gro...,3-3,https://www.e-stat.go.jp/en/stat-search/file-d...,Jul,https://www.e-stat.go.jp/en/stat-search/files?...,2018
5,"Number of Intra-prefectural Migrants, In-migr...",1,https://www.e-stat.go.jp/en/stat-search/file-d...,Aug,https://www.e-stat.go.jp/en/stat-search/files?...,2018
6,Number of Inter-prefectural Migrants by Sex a...,2,https://www.e-stat.go.jp/en/stat-search/file-d...,Aug,https://www.e-stat.go.jp/en/stat-search/files?...,2018
7,Number of In-migrants from Other Prefectures ...,3-1,https://www.e-stat.go.jp/en/stat-search/file-d...,Aug,https://www.e-stat.go.jp/en/stat-search/files?...,2018
8,Number of Out-migrants to Other Prefectures b...,3-2,https://www.e-stat.go.jp/en/stat-search/file-d...,Aug,https://www.e-stat.go.jp/en/stat-search/files?...,2018
9,Number of Net-migration by Age (Five-Year Gro...,3-3,https://www.e-stat.go.jp/en/stat-search/file-d...,Aug,https://www.e-stat.go.jp/en/stat-search/files?...,2018


In [ ]:
#Convert DataFrame to stream and upload to PostgreSQL table on Google Cloud SQL
cur = conn.cursor()
excelurl_textstream = io.StringIO()
upload_df = excelref_df.copy()
upload_df['excel_description'].replace(['\n', '\t'], '', regex=True, inplace=True)

upload_df.to_csv(excelurl_textstream, sep='\t', header=False, index=False, quoting=csv.QUOTE_NONE)
excelurl_textstream.seek(0) 
cur.copy_from(excelurl_textstream, 'jpstat_excel_urls', null="") # null values become ''
conn.commit()

logging.info('Excel URL database table updated.')

#FIXME
# It's neccessary to check the descriptions of the excel files to see if a new description, i.e. new type of excel sheet structure, has been introduced.
# In that scenario we should send a notification or email alert, and write to some log file that a new cleaning method downstream will be required.

exceldesccount_df = excelref_df.groupby(['excel_description']).agg(['count']).sort_values([('year', 'count')], ascending=False).iloc[:,:1]
exceldesccount_df.columns=['count']
exceldesccount_df.reset_index(inplace=True)

exceldesccount_textstream = io.StringIO()
exceldesccount_df.to_csv(exceldesccount_textstream, sep='\t', header=False, index=False)
exceldesccount_textstream.seek(0)
cur.copy_from(exceldesccount_textstream, 'jpstat_excel_descriptions', null="") # null values become ''
conn.commit()

In [3]:
"""
Returns a dataframe of details on excel sheets of data available for a given url
for a particular month
"""
def retrieve_month_excels(murl, path):
    mcols = ['url', 'excel_num', 'excel_description', 'excel_url']
    mdf = pd.DataFrame(columns=mcols)
    mr = re.get(murl)
    mdata = mr.text
    msoup = BeautifulSoup(mdata, "lxml")
    mtable = msoup.find('div', {'class': 'stat-dataset_list-body'})
    for row in mtable.find_all('article', {'class': 'stat-dataset_list-item'}):
        excel_num = row.find('li', {'class': 'stat-dataset_list-detail-item stat-dataset_list-border-top'}).contents[0].replace('\n','')
        excel_description = row.find('a').contents[0]
        excel_url = ''
        excel_a = row.find_all('a')[1]
        
        if(excel_a['data-file_type'] == 'EXCEL'):
            excel_url = path + excel_a['href']
        mdfrow = pd.DataFrame([[murl, excel_num, excel_description, excel_url]], columns=mcols)
        if(len(mdf)==0):
            mdf = mdfrow
        else:
            mdf = mdf.append(mdfrow, ignore_index=True) #why the hell doesn't df.append work inplace?? Didn't it always use to?
    logging.info("Retrieved excel URLs from month-URL: " + murl + '.')
    return(mdf)